<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/alpaca_execution_adapter_LightGBM_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install alpaca-trade-api
!pip install python-dotenv
!pip install ta

In [63]:
import csv
import time
import pickle
import warnings
import pandas as pd
import lightgbm as lgb
import ta
from datetime import datetime, timedelta
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame, APIError

warnings.filterwarnings("ignore")

In [64]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
RESULTS_DIR = "/content/drive/MyDrive/Alpaca_Results/LightGBM"
os.makedirs(RESULTS_DIR, exist_ok=True)


Mounted at /content/drive


In [65]:
from google.colab import files
uploaded = files.upload()  # Prompts you to upload the .env file for


Saving Alpaca_keys.env.txt to Alpaca_keys.env.txt


In [66]:
# Rename to .env so dotenv can recognize it
os.rename("Alpaca_keys.env.txt", ".env")


In [67]:
from google.colab import files
uploaded = files.upload()  # Upload: model_ABT.txt, features_ABT.txt, scaler_ABT.pkl

import lightgbm as lgb
import pickle

print("Model, features, and scaler loaded for ABT.")


Saving features_ABT.txt to features_ABT (5).txt
Saving features_BRK-B.txt to features_BRK-B (5).txt
Saving features_MDT.txt to features_MDT (5).txt
Saving features_PG.txt to features_PG (5).txt
Saving features_PM.txt to features_PM (5).txt
Saving model_ABT.txt to model_ABT (5).txt
Saving model_BRK-B.txt to model_BRK-B (5).txt
Saving model_MDT.txt to model_MDT (5).txt
Saving model_PG.txt to model_PG (5).txt
Saving model_PM.txt to model_PM (5).txt
Saving scaler_ABT.pkl to scaler_ABT (5).pkl
Saving scaler_BRK-B.pkl to scaler_BRK-B (5).pkl
Saving scaler_MDT.pkl to scaler_MDT (5).pkl
Saving scaler_PG.pkl to scaler_PG (5).pkl
Saving scaler_PM.pkl to scaler_PM (5).pkl
Model, features, and scaler loaded for ABT.


In [68]:

#Load .env credentials
load_dotenv(override=True)
API_KEY = os.getenv("APCA_API_KEY_ID")
SECRET_KEY = os.getenv("APCA_API_SECRET_KEY")
BASE_URL = "https://paper-api.alpaca.markets"  # or live URL

#Create API object
api = REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

#Check account
account = api.get_account()
print("Account status:", account.status)

Account status: ACTIVE


In [69]:
#Connect to Alpaca
api = REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')
account = api.get_account()
print("Connected to Alpaca:", account.status)

Connected to Alpaca: ACTIVE


In [70]:

def load_model_artifacts(ticker):
    model = lgb.Booster(model_file=f'model_{ticker}.txt')
    with open(f'features_{ticker}.txt', 'r') as f:
        features = f.read().strip().split(',')
    with open(f'scaler_{ticker}.pkl', 'rb') as f:
        scaler = pickle.load(f)
    return model, features, scaler

def add_features(df):
    df = df.copy()
    df['SMA_50'] = ta.trend.sma_indicator(df['close'], window=50)
    df['EMA_20'] = ta.trend.ema_indicator(df['close'], window=20)

    volatility = df['close'].rolling(14).std()

    rsi = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    macd_diff = ta.trend.macd_diff(df['close'])
    signal_line = ta.trend.macd_signal(df['close'])
    atr = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
    obv = ta.volume.on_balance_volume(df['close'], df['volume'])
    cci = ta.trend.cci(df['high'], df['low'], df['close'], window=20)

    df['RSI_voladj'] = rsi * volatility
    df['MACD_voladj'] = macd_diff * volatility
    df['Signal_Line'] = signal_line
    df['ATR_voladj'] = atr * volatility
    df['OBV'] = obv
    df['CCI_voladj'] = cci * volatility

    return df.dropna()

def log_trade(ticker, signal, action, price, equity, position_qty):
    now = datetime.utcnow().isoformat()
    row = {
        "datetime": now,
        "ticker": ticker,
        "signal": signal,
        "action": action,
        "price": float(price),
        "equity": float(equity),
        "position_qty": float(position_qty)
    }
    log_file = os.path.join(RESULTS_DIR, f"trade_log_{ticker}.csv")
    file_exists = os.path.isfile(log_file)
    with open(log_file, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)
    print(f"Logged trade to Drive: {log_file}")

In [71]:
tickers = ["ABT", "BRK.B", "PM", "PG", "MDT"]

for ticker in tickers:
    print(f"\nRunning strategy for {ticker}...")

    try:
        model, model_features, scaler = load_model_artifacts(ticker)
        print(f"Model artifacts loaded for {ticker}")
    except Exception as e:
        print(f"Could not load model for {ticker}: {e}")
        continue

    try:
        end = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
        start = (datetime.utcnow() - timedelta(days=45)).replace(microsecond=0).isoformat() + "Z"
        bars = api.get_bars(ticker, TimeFrame.Hour, start=start, end=end, feed="iex").df
        bars = bars.reset_index()
        bars = add_features(bars)
        if len(bars) < 10:
            print(f"Not enough data after features for {ticker}")
            continue
    except Exception as e:
        print(f"Error fetching/processing bars for {ticker}: {e}")
        continue

    if not all(f in bars.columns for f in model_features):
        missing = [f for f in model_features if f not in bars.columns]
        print(f"Missing features for {ticker}: {missing}")
        continue

    # Prediction
    X_live = bars[model_features].copy().dropna()
    X_scaled = scaler.transform(X_live)
    preds = model.predict(X_scaled)
    signal = int(preds[-1])
    print(f"Prediction for {ticker}: {signal} (1 = Buy, 0 = Sell)")

    # Trade Execution
    price = bars.iloc[-1]["close"]
    try:
        history = api.get_portfolio_history(period="1D", timeframe="5Min")
        equity = history.equity[-1]
    except:
        equity = 0

    action = "Hold"
    position_qty = 0

    try:
        clock = api.get_clock()
        if not clock.is_open:
            print("Market is closed.")
            continue

        try:
            position = api.get_position(ticker)
            has_position = float(position.qty) > 0
            position_qty = float(position.qty)
        except APIError:
            has_position = False
            position_qty = 0
            print(f"No current position in {ticker}")

        if signal == 1 and not has_position:
            api.submit_order(symbol=ticker, qty=1, side='buy', type='market', time_in_force='gtc')
            action = "Buy"
            print(f"BUY order submitted for {ticker}")
        elif signal == 0 and has_position:
            api.submit_order(symbol=ticker, qty=1, side='sell', type='market', time_in_force='gtc')
            action = "Sell"
            print(f"SELL order submitted for {ticker}")
        else:
            print(f"No action taken for {ticker}")

    except Exception as e:
        print(f"Trade error for {ticker}: {e}")
        continue

    log_trade(ticker, signal, action, price, equity, position_qty)

    print("Portfolio snapshot:")
    positions = api.list_positions()
    for p in positions:
        print(f"{p.symbol}: {p.qty} shares (${p.market_value})")

    print("Recent Equity:", history.equity[-3:])
    print(f"Strategy complete for {ticker}")


Running strategy for ABT...
Model artifacts loaded for ABT
Prediction for ABT: 0 (1 = Buy, 0 = Sell)
No current position in ABT
No action taken for ABT
Logged trade to Drive: /content/drive/MyDrive/Alpaca_Results/LightGBM/trade_log_ABT.csv
Portfolio snapshot:
AAPL: 2 shares ($421.3998)
Recent Equity: [100012.48, 100012.64, 100012.64]
Strategy complete for ABT

Running strategy for BRK.B...
Could not load model for BRK.B: Could not open model_BRK.B.txt

Running strategy for PM...
Model artifacts loaded for PM
Prediction for PM: 0 (1 = Buy, 0 = Sell)
No current position in PM
No action taken for PM
Logged trade to Drive: /content/drive/MyDrive/Alpaca_Results/LightGBM/trade_log_PM.csv
Portfolio snapshot:
AAPL: 2 shares ($421.3996)
Recent Equity: [100012.48, 100012.62, 100012.62]
Strategy complete for PM

Running strategy for PG...
Model artifacts loaded for PG
Prediction for PG: 0 (1 = Buy, 0 = Sell)
No current position in PG
No action taken for PG
Logged trade to Drive: /content/drive/M

In [72]:
#Log Portfolio and Account Status
positions = api.list_positions()
for p in positions:
    print(f"{p.symbol}: {p.qty} shares (${p.market_value})")

history = api.get_portfolio_history(period="1D", timeframe="5Min")
print("Recent Equity:", history.equity[-5:])


print("Strategy complete. Await next live cycle.")


AAPL: 2 shares ($421.3996)
Recent Equity: [100011.54, 100012.02, 100012.48, 100012.64, 100012.64]
Strategy complete. Await next live cycle.
